In [1]:
import numpy as np
import pandas as pd

import scipy.spatial.distance

## Read reference

From Table 3

In [2]:
df_t3 = pd.read_excel("./digitized_tables.xlsx", sheet_name='Table 3')
df_t3.index = df_t3.index + 1

names = df_t3['Organism']
names = names.iloc[:13].str.split(n=1).str[1]

df_t3.drop(columns=['Organism'], index=14, inplace=True)
df_t3 = df_t3.apply(lambda s: s.replace('NaN', '').replace('-', np.nan)).astype(float)

df_t3.iloc[:10, :10]

,1,2,3,4,5,6,7,8,9,10
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.60,0.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.50,0.48,0.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.53,0.49,0.51,0.60,NaN,NaN,NaN,NaN,NaN,NaN
6,0.52,0.49,0.51,0.54,0.60,NaN,NaN,NaN,NaN,NaN
7,0.25,0.27,0.25,0.26,0.23,0.25,NaN,NaN,NaN,NaN
8,0.26,0.28,0.26,0.28,0.27,0.29,0.59,NaN,NaN,NaN
9,0.20,0.24,0.21,0.23,0.23,0.22,0.51,0.52,NaN,NaN
10,0.29,0.26,0.24,0.24,0.26,0.25,0.33,0.41,0.34,NaN


In [3]:
names

1               M. arbophilicum
2             M. ruminantium PS
3            M. ruminantium M-1
4                 M. formicicum
5                 M. sp. M.o.H.
6        M. thermoautotrophicum
7          Cariaco isolate JR-1
8        Black Sea isolate JR-1
9     Methanospirillum hungatii
10       Methanosarcina barkeri
11           Enteric-vibrio sp.
12                 Bacillus sp.
13               Blue-green sp.
Name: Organism, dtype: object

## Read data

Digitized data from Table 1

In [4]:
df = pd.concat([pd.read_excel("./digitized_tables.xlsx", sheet_name=n, dtype=str)
    for n in ['Table 1 (p4538)', 'Table 1 (p4539)']])
df.columns = 3 * ['seq', 'org']

df = pd.concat([df.iloc[:, 0:2], df.iloc[:, 2:4], df.iloc[:, 4:6]], ignore_index=False, sort=False)
df.dropna(inplace=True)

def parse_org(s):

    tokens = s.replace(';', ',').split(',')
    org = []
    for t in tokens:
        if '-' in t:
            a, b = t.split('-')
            a = int(a)
            b = int(b)
            org.extend(range(a, b + 1))
        else:
            org.append(int(t))
    return org

df = df.set_index('seq')
df = df['org'].apply(parse_org).explode()

df = df.reset_index()

In [5]:
df_feat = df.groupby(['seq', 'org']).apply(lambda x: x.shape[0]).unstack('org')
df_feat = df_feat.fillna(0).astype(int)
df_feat.head()

org,1,2,3,4,5,6,7,8,9,10
seq,,,,,,,,,,
"(AAACA,UAAUCUCA)-CCCAUCCUUAG",0,0,0,0,0,0,0,0,0,1
"(CAA,CCA)CAUUCUG",0,0,0,0,0,1,0,0,0,0
"(CCA,CAA)CAG",0,0,0,0,0,0,0,1,0,0
"(CU,CCUU)CG",0,0,0,1,0,0,0,0,0,0
"(CUA,CUUUUA)UUG",0,0,1,0,0,0,0,0,0,0


In [6]:
import itertools

def dist_mtx(df, func, **kwargs):
    n_col = len(df.columns)
    df_dist = pd.DataFrame(np.zeros([n_col, n_col], dtype=float),
        columns=df.columns, index=df.columns)

    # off diagonal
    for c1, c2 in itertools.combinations(df.columns, 2):
         df_dist.loc[c1, c2] = df_dist.loc[c2, c1] = \
            func(df[c1], df[c2], **kwargs)

    # on diagonal
    # for c in df.columns:
    #     df_dist.loc[c, c] = func(df[c], df[c], **kwargs)

    return df_dist

def clean_idx(x):
    return  x.index.str.replace('[^ACTGU]', '')

def woese_assoc(v1, v2):
    """Each oligo binary -- counted or not"""
    seqs_len = clean_idx(v1).str.len().values

    Na = seqs_len * (v1 > 0)
    Na = Na.sum()
    Nb = seqs_len * (v2 > 0)
    Nb = Nb.sum()

    Nab = seqs_len[(v1 > 0) & (v2 > 0)].sum()
    return 2 * Nab / (Na + Nb)

def woese_assoc_2(v1, v2):
    """Each oligo proportional to count"""
    seqs_len = clean_idx(v1).str.len().values

    Na = seqs_len * v1
    Na = Na.sum()
    Nb = seqs_len * v2
    Nb = Nb.sum()

    v_min = pd.concat([v1, v2], axis=1)
    v_min = v_min.min(axis=1)
    Nab = seqs_len * v_min
    Nab = Nab.sum()

    return 2 * Nab / (Na + Nb)

def dice_coefficient(a, b):
    """dice coefficient 2nt/(na + nb).
    https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Dice%27s_coefficient#Python
    """
    a_bigrams = set(a[a > 0].index.tolist())
    b_bigrams = set(b[b > 0].index.tolist())
    overlap = len(a_bigrams & b_bigrams)
    return overlap * 2.0/(len(a_bigrams) + len(b_bigrams))

def dice_sim(v1, v2):
    return 1 - scipy.spatial.distance.dice(v1 > 0, v2 > 0)

def dice_sim_wt(v1, v2):
    wt = clean_idx(v1).str.len().values
    return 1 - scipy.spatial.distance.dice(v1 > 0, v2 > 0, wt)

In [7]:
oligo_len = clean_idx(df_feat).str.len()
pd.options.display.float_format = lambda x: '{:.4f}'.format(x).lstrip('0')

df_dice_wt = dist_mtx(df_feat[oligo_len >= 6], dice_sim_wt)
df_dice_wt.round(2).values - df_t3.iloc[:10, :10]

,1,2,3,4,5,6,7,8,9,10
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,-0.0100,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,.0100,.0100,nan,nan,nan,nan,nan,nan,nan,nan
4,.0000,-0.0200,-0.0300,nan,nan,nan,nan,nan,nan,nan
5,-0.0100,-0.0100,-0.0300,-0.0100,nan,nan,nan,nan,nan,nan
6,-0.0100,-0.0100,-0.0300,.0000,-0.0100,nan,nan,nan,nan,nan
7,-0.0100,.0000,.0000,.0100,.0000,.0100,nan,nan,nan,nan
8,.0000,.0000,.0000,.0000,.0000,.0100,-0.0100,nan,nan,nan
9,.0100,.0100,.0100,.0000,.0100,.0300,-0.0200,-0.0300,nan,nan
10,-0.0200,.0300,.0400,.0200,.0200,.0700,-0.0200,-0.0100,-0.0200,nan


## Calculating S_ab by presence/absence of a given oligo

In [8]:
df_woese = dist_mtx(df_feat[oligo_len >= 6], woese_assoc)
df_woese.values - df_t3.iloc[:10, :10]

,1,2,3,4,5,6,7,8,9,10
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,-0.0107,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,.0129,.0114,nan,nan,nan,nan,nan,nan,nan,nan
4,-0.0026,-0.0194,-0.0290,nan,nan,nan,nan,nan,nan,nan
5,-0.0080,-0.0115,-0.0309,-0.0077,nan,nan,nan,nan,nan,nan
6,-0.0054,-0.0066,-0.0330,-0.0040,-0.0131,nan,nan,nan,nan,nan
7,-0.0057,-0.0008,-0.0037,.0057,-0.0030,.0051,nan,nan,nan,nan
8,.0007,.0027,.0033,-0.0004,-0.0045,.0093,-0.0128,nan,nan,nan
9,.0138,.0091,.0067,.0006,.0117,.0250,-0.0184,-0.0281,nan,nan
10,-0.0186,.0326,.0375,.0190,.0215,.0708,-0.0223,-0.0066,-0.0238,nan


## Calculating S_ab by the number of overlapping oligos

For example, if a given oligo occurs thrice in 1 and twice in 2, count it twice.

In [9]:
df_woese = dist_mtx(df_feat[oligo_len >= 6], woese_assoc_2)
df_woese.values - df_t3.iloc[:10, :10]

,1,2,3,4,5,6,7,8,9,10
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,-0.0171,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,.0067,.0114,nan,nan,nan,nan,nan,nan,nan,nan
4,-0.0077,-0.0194,-0.0290,nan,nan,nan,nan,nan,nan,nan
5,-0.0134,-0.0115,-0.0309,-0.0077,nan,nan,nan,nan,nan,nan
6,-0.0106,-0.0066,-0.0330,-0.0040,-0.0131,nan,nan,nan,nan,nan
7,-0.0082,-0.0008,-0.0037,.0057,-0.0030,.0051,nan,nan,nan,nan
8,-0.0032,.0012,.0019,-0.0019,-0.0060,.0077,-0.0158,nan,nan,nan
9,.0105,.0078,.0055,-0.0006,.0104,.0237,-0.0210,-0.0227,nan,nan
10,-0.0226,.0312,.0361,.0177,.0201,.0691,-0.0239,-0.0107,-0.0271,nan


#### Sandbox below...

In [10]:
(df_dice_wt.round(2).values - df_t3.iloc[:10, :10]) - (df_woese.values - df_t3.iloc[:10, :10])

,1,2,3,4,5,6,7,8,9,10
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,.0071,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,.0033,-0.0014,nan,nan,nan,nan,nan,nan,nan,nan
4,.0077,-0.0006,-0.0010,nan,nan,nan,nan,nan,nan,nan
5,.0034,.0015,.0009,-0.0023,nan,nan,nan,nan,nan,nan
6,.0006,-0.0034,.0030,.0040,.0031,nan,nan,nan,nan,nan
7,-0.0018,.0008,.0037,.0043,.0030,.0049,nan,nan,nan,nan
8,.0032,-0.0012,-0.0019,.0019,.0060,.0023,.0058,nan,nan,nan
9,-0.0005,.0022,.0045,.0006,-0.0004,.0063,.0010,-0.0073,nan,nan
10,.0026,-0.0012,.0039,.0023,-0.0001,.0009,.0039,.0007,.0071,nan


In [11]:
df_dice = dist_mtx(df_feat[oligo_len >= 6], dice_sim)
df_dice.round(2).values #- df_t3.iloc[:10, :10]

array([[0.  , 0.67, 0.63, 0.53, 0.56, 0.54, 0.26, 0.28, 0.23, 0.3 ],
       [0.67, 0.  , 0.63, 0.49, 0.52, 0.52, 0.3 , 0.32, 0.28, 0.33],
       [0.63, 0.63, 0.  , 0.49, 0.51, 0.51, 0.27, 0.29, 0.24, 0.31],
       [0.53, 0.49, 0.49, 0.  , 0.63, 0.57, 0.29, 0.31, 0.26, 0.29],
       [0.56, 0.52, 0.51, 0.63, 0.  , 0.63, 0.25, 0.3 , 0.27, 0.31],
       [0.54, 0.52, 0.51, 0.57, 0.63, 0.  , 0.28, 0.32, 0.27, 0.35],
       [0.26, 0.3 , 0.27, 0.29, 0.25, 0.28, 0.  , 0.63, 0.52, 0.33],
       [0.28, 0.32, 0.29, 0.31, 0.3 , 0.32, 0.63, 0.  , 0.55, 0.43],
       [0.23, 0.28, 0.24, 0.26, 0.27, 0.27, 0.52, 0.55, 0.  , 0.35],
       [0.3 , 0.33, 0.31, 0.29, 0.31, 0.35, 0.33, 0.43, 0.35, 0.  ]])

In [12]:
df_dice = dist_mtx(df_feat[oligo_len >= 6], dice_coefficient)
df_dice.round(2).values  # - df_t3.iloc[:10, :10]

array([[0.  , 0.67, 0.63, 0.53, 0.56, 0.54, 0.26, 0.28, 0.23, 0.3 ],
       [0.67, 0.  , 0.63, 0.49, 0.52, 0.52, 0.3 , 0.32, 0.28, 0.33],
       [0.63, 0.63, 0.  , 0.49, 0.51, 0.51, 0.27, 0.29, 0.24, 0.31],
       [0.53, 0.49, 0.49, 0.  , 0.63, 0.57, 0.29, 0.31, 0.26, 0.29],
       [0.56, 0.52, 0.51, 0.63, 0.  , 0.63, 0.25, 0.3 , 0.27, 0.31],
       [0.54, 0.52, 0.51, 0.57, 0.63, 0.  , 0.28, 0.32, 0.27, 0.35],
       [0.26, 0.3 , 0.27, 0.29, 0.25, 0.28, 0.  , 0.63, 0.52, 0.33],
       [0.28, 0.32, 0.29, 0.31, 0.3 , 0.32, 0.63, 0.  , 0.55, 0.43],
       [0.23, 0.28, 0.24, 0.26, 0.27, 0.27, 0.52, 0.55, 0.  , 0.35],
       [0.3 , 0.33, 0.31, 0.29, 0.31, 0.35, 0.33, 0.43, 0.35, 0.  ]])